# Wrangle Report

This reports describes the wrangling efforts and procedure used to clean the datasets.

## Gathering

The dataset was gather using three methods. 

In the first method, I downloaded the twitter-archived-enhanced.csv from the Udacity website. This was then read into a Panda dataframe using the read_csv() function.

The second dataset was downloaded directly into my notebook using an URL and the Request library. The dataset was a tab seperated file called image-predictionand this was read into a dataframe using Panda's read_csv() function.

The final dataset was queried directly from Twitter's API in JSON format using the Tweepy library. This was then stored as  text file. I however had problems with mobile verification and used the provided tweet_json.txt text file. This text file was the read in line-by-line into list and conerted to a dataframe.

## Assessing

The data was assessed in two ways: visual assessment and programmatic assessment. 

For visual assessment, I displayed each dataset using an external application and scrolled through it to understand and find any errors.

Programmtic assessment was handled in two ways: through a non-directed approach and a direct approach. For the non-directed approach I used Panda's sample() function a multiple times to find an issues.

In the directed approach, I used the info() function to evaluate the non-null counts and data types. I also used describe() function to evaluate the numeric data types.

Based on these techniques, the following tidiness and quality issues were determined.

Tidiness:

1. twitter-archived-enhanced table:
    1. The majority of values for `in_reply_to_status_id` and `in_reply_to_user_id` are null. This might not be able to be addressed.
    2. Some dogs are named to 'a' and all lowercase names are not dog names
    3. `timestamp` is a string type and not a dateTime type.
    4. Some `rating_denominator` values aren't 10. All should be equal to 10.
    5. Some rows are retweets (not original ratings) marked by the `retweeted_status_id` column.
    6. Not all tweets have images marked by nulls in `expanded_url` column.
    7. `tweet_id` should have string datatype 
    
2. image-predictions table:
    1. Column headers could be more descriptive
    2. Dog prediction doesn't always predict a dog.

Quality:

3. twitter-archived-enhanced table:
    1. The dog stages are seperated into 4 different columns. This could aggregated as one dog stage column. It should be of Dtype unordered catergorical
4. tweet-json table:
    1. This table contains almost all the same tweet ids as the 'twitter-archived-enhanced' table. This should be merged into one table.



## Cleaning

Data cleaning followed the Define-Code-Test cleaning process. However, before this was implemented, I made a copy of the dataframes as to prevent modifications to the original datasets.

Issue 1.3 and 1.7 was fixed first by casting the `timestamp` column to dateTime data type using Panda's to_datetime() function and `tweet_id` columns was cast to string type using the astype() function.

I then handled issue 1.1. These data values were mostly null and not necessary to address as they could be valid entries as not necessarily every tweet is a reply. I chose to drop these columns using the Panda's drop function. 

Issue 1.2 was addressed aferwards. I first checked if the name is the same if converted to lowercase. I then replaced all these lowercasen names with NaN values and then replaced the NaN value with None. 

I then addressed issue 4.1. I merged the two dataframes `df_twitter_archive_enhanced_copy` and `df_tweet_json_copy` into one dataframe called `df_merged` on the tweet IDs using the Panda's merge function. I used the intersection of the two tweet IDs to ensure there was no missing values.

Issue 3.1 was addressed afterwards. I first replaced all the None string values to None null type. I then created a new column called 'dog_stages' and used numpy's select() function to condition the choice. If there was no dog stage, it was set to the default of None. O then cast the column to an unordered categroical data type.

Issue 1.6 was then handled by using Panda's dropna() on the subset column `expanded_urls` to remove rows with tweet IDs that have no image.

I now addressed issue 1.5. The specifications required only original tweets, no retweets. To do this, I filtered the dataframe by all the rows where the values are null for the column `retweeted_status_id`. This corresponds to original tweets only. I then dropped the columns `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp`.

Issue 1.4 was addressed next. I replaced all the denominators in `rating_denominator` to the value of 10 using the replace() function. 

Issue 2.1 was now addressed. I used pandas's rename() function to change the column names in image-predictions to a more descriptive name.

Finally, issue 2.3 was addressed. First, I isolated the `prediction_1_is_a_breed_of_dog`, `prediction_2_is_a_breed_of_dog` and `prediction_3_is_a_breed_of_dog` which indicates whether the predictions are true or false for a dog. I then removed the rows if all three columns were false by evlauting them using Panda's any() function. A column called `dog_predictions_results` was created which contained these evaluations. I then filtered out the dataframe using this column and dropped the `dog_predictions_results` column. I then used numpy's select() function to choose the first dog breed that was true for the image predictions an created a new column called `dog_breed`. Afer which I dropped the unnecessary columns for the predictions, confidence scores and boolean results for dog breed per prediction.



## Data Storage

The cleaned dataframes was then stored as a comma seperated value file called `twitter_archive_master.csv` and `image_predictions_clean.csv` using Panda's to_csv() function. 